In [ ]:
# 400 meter coordinate accuracy
# 1924 earliest year
import pandas as pd
import geopandas as gpd

# from dask import dataframe as dd

# import dask_geopandas

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
# read in cleaned parquet
df_path = input("Enter the file path: ")

# df = dd.read_parquet(df_path)
# df = gpd.read_parquet(df_path)
df = pd.read_parquet(df_path)

In [ ]:
# read in eco_map.geojson
map_path = input("Enter the file path: ")

map = gpd.read_file(map_path)

In [ ]:
map.head()

In [ ]:
map.crs

In [ ]:
df = gpd.GeoDataFrame(df)

In [ ]:
df["geometry"] = gpd.points_from_xy(df["decimalLongitude"], df["decimalLatitude"])

In [ ]:
df = df.set_geometry("geometry")

df = df.set_crs("EPSG:4326")

In [ ]:
df.crs

In [ ]:
df = df.drop(["decimalLatitude", "decimalLongitude"], axis=1)

In [ ]:
df = df.sjoin(map, predicate="within")

In [ ]:
df

In [ ]:
# ecoregions
# remove rows with <NA> in unique_id from df
df = df[df["unique_id"] != "<NA>"]

# freshwater
# df = df[df["id"] != "<NA>"]

# soil
# df = df[df["id"] != "<NA>"]

In [ ]:
del map, map_path

In [ ]:
df

In [ ]:
# ecoregions
df = df.drop(["index_right", "geometry", "name", "TYPE"], axis=1)

# freshwater
# df = df.drop(["index_right", "geometry", "name"], axis=1)

# soil
# df = df.drop(
#     [
#         "index_right",
#         "geometry",
#         "dominant_soil_type_percentage",
#         "soil_texture",
#         "soil_slope",
#         "soil_id",
#         "FAOSOIL",
#     ],
#     axis=1,
# )

In [ ]:
df

In [ ]:
# ecoregions
df = df.groupby(
    ["kingdom", "phylum", "class", "order", "family", "genus", "species"],
    as_index=False,
    dropna=False,
).apply(
    lambda x: pd.Series(
        {
            "unique_id": list(x["unique_id"].unique()),
            "rights": list(
                x[["unique_id", "rights"]].drop_duplicates("unique_id", keep="first")[
                    "rights"
                ]
            ),
        }
    )
)

# freshwater
# df = df.groupby(
#     ["kingdom", "phylum", "class", "order", "family", "genus", "species"],
#     as_index=False, dropna=False,
# ).apply(
#     lambda x: pd.Series(
#         {
#             "id": list(x["id"].unique()),
#         }
#     )
# )

# soil
# df = df.groupby(
#     ["kingdom", "phylum", "class", "order", "family", "genus", "species"],
#     as_index=False, dropna=False,
# ).apply(
#     lambda x: pd.Series(
#         {
#             "id": list(x["id"]),
#             "specific_soil_name": list(x["specific_soil_name"]),
#             "dominant_soil_name": list(x["dominant_soil_name"]),
#         }
#     )
# )

In [ ]:
# ECOREGIONS ONLY
# print max length of unique_id and rights
print(df["unique_id"].str.len().max())
print(df["rights"].str.len().max())

In [ ]:
# ECOREGIONS ONLY
# find rows where length of unique_id !equals length of rights (shouldn't happen)
df[df["rights"].str.len() != df["unique_id"].str.len()]

In [ ]:
duplicates = df[df.duplicated(["species"], keep=False)]

duplicates

In [ ]:
df

In [ ]:
# ecoregions
# df = df.rename(
#     columns={"species": "scientific_name", "unique_id": "observed_ecoregions"}
# )

# freshwater
# df = df.rename(columns={"species": "scientific_name", "id": "freshwater_ecoregions"})

# soil
df = df.rename(columns={"species": "scientific_name", "id": "soil_id"})

In [ ]:
df

In [ ]:
df_spatial_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
df.to_parquet(df_spatial_path)